- Classification and flowers
- Florent Dufour
- 2016 - 2022

In [1]:
import tensorflow as tf
import pandas as pd

In [19]:
# Get datasets

CSV_COLUMN_NAMES = ["SepalLength", "SepalWidth", "PetalLength", "PetalWidth", "Species"]
SPECIES = ["Setosa", "Versicolor", "Virginica"]

train = pd.read_csv("https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv",
  names = CSV_COLUMN_NAMES,
  header = 0)

test = pd.read_csv("https://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv",
  names = CSV_COLUMN_NAMES,
  header = 0)

train.head()

,SepalLength,SepalWidth,PetalLength,PetalWidth,Species
0,6.4,2.8,5.6,2.2,2
1,5.0,2.3,3.3,1.0,1
2,4.9,2.5,4.5,1.7,2
3,4.9,3.1,1.5,0.1,0
4,5.7,3.8,1.7,0.3,0


In [20]:
# Prepare the dataset
train_y = train.pop("Species")
test_y = test.pop("Species")
train_y.head()

,SepalLength,SepalWidth,PetalLength,PetalWidth
0,6.4,2.8,5.6,2.2
1,5.0,2.3,3.3,1.0
2,4.9,2.5,4.5,1.7
3,4.9,3.1,1.5,0.1
4,5.7,3.8,1.7,0.3


In [21]:
train.shape

(120, 4)

In [30]:
# Input function

def input_fn(features, labels, training=True, batch_size=256):
  dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))
  if training:
    dataset = dataset.shuffle(1000).repeat()
  return dataset.batch(batch_size)

In [31]:
# Features column

my_feature_colums = []
for key in train.keys():
  my_feature_colums.append(tf.feature_column.numeric_column(key=key))
print(len(my_feature_colums))

4


In [32]:
# Build the DNN

classifier = tf.estimator.DNNClassifier(
    feature_columns=my_feature_colums,
    hidden_units = [30,10], # 2 hidden layers: 30 and 10 nodes respectively
    n_classes = 3)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmp956je_vo', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [34]:
# Train the classifier

classifier.train(
  input_fn = lambda: input_fn(train, train_y, training=True),
  steps = 10000
)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp956je_vo/model.ckpt-5000
Instructions for updating:
Use standard file utilities to get mtimes.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 5000...
INFO:tensorflow:Saving checkpoints for 5000 into /tmp/tmp956je_vo/model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 5000...
INFO:tensorflow:loss = 0.48378405, step = 5000
INFO:tensorflow:global_step/sec: 386.437
INFO:tensorflow:loss = 0.4689865, step = 5100 (0.264 sec)
INFO:tensorflow:global_step/sec: 473.175
INFO:tensorflow:loss = 0.464562, step = 5200 (0.210 sec)
INFO:tensorflow:global_step/sec: 425.865
INFO:tensorflow:loss = 0.45639533, step = 5300 (0.232 sec)
INFO:tensorflow:global_step/sec: 505.297


In [38]:
classifier.evaluate(input_fn=lambda: input_fn(test, test_y, training=False))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2022-01-21T09:43:14
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp956je_vo/model.ckpt-15000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 0.26854s
INFO:tensorflow:Finished evaluation at 2022-01-21-09:43:15
INFO:tensorflow:Saving dict for global step 15000: accuracy = 0.93333334, average_loss = 0.3101095, global_step = 15000, loss = 0.3101095
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 15000: /tmp/tmp956je_vo/model.ckpt-15000


{'accuracy': 0.93333334,
 'average_loss': 0.3101095,
 'global_step': 15000,
 'loss': 0.3101095}

In [52]:
# Predict

## Input values
features = ["SepalLength", "SepalWidth", "PetalLength", "PetalWidth"]
predict = {}
print("Please type numeric values as prompted.")
for feature in features:
  val = input(feature + ": ")
  predict[feature] = [float(val)]

## Make prediction

def input_fn(features, batch_size=256):
  return tf.data.Dataset.from_tensor_slices(dict(features)).batch(batch_size)

predictions = classifier.predict(input_fn = lambda: input_fn(predict))
for pred_dict in predictions:
  class_id = pred_dict["class_ids"][0]
  probability = pred_dict["probabilities"][class_id]

  print('Prediction is "{}" ({:.1f}%)'.format(SPECIES[class_id], 100*probability))

Please type numeric values as prompted.
SepalLength: 2.4
SepalWidth: 2.6
PetalLength: 6.5
PetalWidth: 6.3
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp956je_vo/model.ckpt-15000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Prediction is "Virginica" (96.6%)


In [55]:
# Make preduction bis

# Here is some example input and expected classes you can try above
expected = ['Setosa', 'Versicolor', 'Virginica']
predict = {
    'SepalLength': [5.1, 5.9, 6.9],
    'SepalWidth': [3.3, 3.0, 3.1],
    'PetalLength': [1.7, 4.2, 5.4],
    'PetalWidth': [0.5, 1.5, 2.1],
}

predictions = classifier.predict(input_fn = lambda: input_fn(predict))
for pred_dict in predictions:
  class_id = pred_dict["class_ids"][0]
  probability = pred_dict["probabilities"][class_id]

  print('Prediction is "{}" ({:.1f}%)'.format(SPECIES[class_id], 100*probability))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp956je_vo/model.ckpt-15000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Prediction is "Setosa" (92.8%)
Prediction is "Versicolor" (62.8%)
Prediction is "Virginica" (68.5%)
